In [6]:
from xgboost import XGBRegressor as XGBR 
from xgboost import  XGBClassifier as XGBC 
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.linear_model import LinearRegression as LinearR 
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold,cross_val_score as CVS,train_test_split as TTS
from sklearn.metrics import mean_squared_error as MSE
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt  
from sklearn.datasets import make_blobs
from sklearn.metrics import confusion_matrix as cm,recall_score as recall,roc_auc_score as auc


class_1 = 500 # 类别1 有500个样本
class_2 = 50 # 类别2 有50个样本

centers = [[0.0,0.0],[2.0,2.0]]  # 设定两个类别中心
clusters_std = [1.5,0.5] # 设定两个类别的方差，通常来说，样本量比较大的类别会比较松散

x,y = make_blobs(n_samples =[class_1,class_2],centers=centers,cluster_std=clusters_std,
                random_state =0,shuffle =False)

xtrain,xtest,ytrain,ytest = TTS(x,y,test_size =0.3,random_state = 420)

(y ==1).sum() /y.shape[0]


0.09090909090909091

In [11]:
x.shape

(550, 2)

In [12]:
y.shape

(550,)

In [13]:
clf = XGBC().fit(xtrain,ytrain)
ypred = clf.predict(xtest)


In [14]:
ypred

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [15]:
cm(ytest,ypred,labels=[1,0])

array([[  9,   4],
       [  8, 144]], dtype=int64)

In [16]:
recall(ytest,ypred)

0.6923076923076923

In [17]:
auc(ytest,clf.predict_proba(xtest)[:,1])

0.9671052631578947

In [18]:
# 正负样本比例
clf_ = XGBC(scale_pos_weight=10).fit(xtrain,ytrain)
ypred_ = clf_.predict(xtest)

clf_.score(xtest,ytest)

0.9515151515151515

查看随着样本权重逐渐增加，模型的recall，auc和准确率如何变化

In [19]:
for i in [1,5,10,20,30]:
    clf = XGBC(scale_pos_weight=i).fit(xtrain,ytrain)
    ypred_ = clf_.predict(xtest)
    print(i)
    print("\tAccuracy:{}".format(clf_.score(xtest,ytest)))
    print("\tRecall:{}".format(recall(ytest,ypred_)))
    print("\tAUC:{}".format(auc(ytest,clf_.predict_proba(xtest)[:,1])))

1
	Accuracy:0.9515151515151515
	Recall:1.0
	AUC:0.9696356275303644
5
	Accuracy:0.9515151515151515
	Recall:1.0
	AUC:0.9696356275303644
10
	Accuracy:0.9515151515151515
	Recall:1.0
	AUC:0.9696356275303644
20
	Accuracy:0.9515151515151515
	Recall:1.0
	AUC:0.9696356275303644
30
	Accuracy:0.9515151515151515
	Recall:1.0
	AUC:0.9696356275303644


# 使用xgboots本身自带的库

In [21]:
import xgboost as xgb 

dtrain = xgb.DMatrix(xtrain,ytrain)
dtest = xgb.DMatrix(xtest,ytest)

In [23]:
param = {"silent":True
          ,"obj":"binary:logistic"
          ,"eta":0.1
          ,"scale_pos_weight":1}

num_round = 100

In [25]:
bst = xgb.train(param,dtrain,num_round)

preds = bst.predict(dtest)

In [26]:
# 看看preds 返回了什么
preds 

array([-2.1195412e-04,  2.8303564e-03, -2.1195412e-04, -2.1195412e-04,
        6.1283749e-01,  3.2312274e-03, -2.1195412e-04, -2.1195412e-04,
       -2.1195412e-04, -2.1195412e-04, -2.1195412e-04,  2.6429296e-03,
        1.5855670e-02,  6.5388292e-02,  2.5006831e-03, -2.1195412e-04,
       -2.1195412e-04, -2.1195412e-04, -2.1195412e-04, -2.1195412e-04,
       -2.1195412e-04, -2.1195412e-04, -2.1195412e-04, -2.1195412e-04,
       -2.1195412e-04,  6.3392520e-04, -2.1195412e-04, -2.1195412e-04,
       -2.1195412e-04, -2.1195412e-04, -2.1195412e-04, -2.1195412e-04,
       -2.1195412e-04,  1.7589629e-03,  1.1099279e-03, -2.1195412e-04,
       -7.3075294e-03, -2.9118598e-02, -2.1195412e-04,  3.2312274e-03,
       -1.0898113e-03, -9.6875429e-03, -2.1195412e-04, -2.1195412e-04,
       -2.1195412e-04,  1.3885796e-03,  1.0616183e-03, -2.1195412e-04,
        8.5341930e-04,  1.1099279e-03, -2.1195412e-04, -2.1195412e-04,
        1.6975701e-03, -2.1195412e-04, -2.1195412e-04, -2.1195412e-04,
      

In [27]:
# 自己设定的阈值

ypred = preds.copy()
ypred[preds>0.5] =1 
ypred[ypred !=1] =0 

In [31]:
# 写明参数

scale_pos_weight = [1,5,10]
names = ["negative vs positive: 1","negative vs positive:5","negative vs positive :10"]

In [34]:
# 导入模型评估指标
from sklearn.metrics import accuracy_score as accuracy,recall_score as recall,roc_auc_score as auc

for name,i in zip(names,scale_pos_weight):
    param = {"silent":True
          ,"obj":"binary:logistic"
          ,"eta":0.1
          ,"scale_pos_weight":i}
    num_round = 100
    clf = xgb.train(param,dtrain,num_round)
    ypreds = clf.predict(dtest)
    ypred = preds.copy()
    ypred[preds>0.5] =1 
    ypred[ypred !=1] =0 
    print(name)
    print("\tAccuracy:{}".format(accuracy(ytest,ypred)))
    print("\tRecall:{}".format(recall(ytest,ypred)))
    print("\tAUC:{}".format(auc(ytest,preds)))
    

negative vs positive: 1
	Accuracy:0.9272727272727272
	Recall:0.6923076923076923
	AUC:0.9731781376518218
negative vs positive:5
	Accuracy:0.9272727272727272
	Recall:0.6923076923076923
	AUC:0.9731781376518218
negative vs positive :10
	Accuracy:0.9272727272727272
	Recall:0.6923076923076923
	AUC:0.9731781376518218
